# Comparison of Recognition Gaps

This notebook creates a bar plot figure of the magnitudes and standard deviations of recognition gaps for different experiments. In the manuscript, it is Figure 3B.

The figure displays two vertical bars for the recognition gaps
evaluated with BagNet-33 on the data from Ullman et al. (2016) and on
ImageNet data (Deng et al., 2009). Furthermore, it shows the values
found by Ullman et al. (2016) for both machine algorithms (also a
vertical bar) as well as for humans (horizonal bar).

## Your TODO

Please specify the path to the most top directory of your recognition
gap experiments, i.e. the parent directory of the analysis folder.

In [ ]:
import plot_utils
import data_csv_utils
import sys
import matplotlib.pyplot as plt
path_to_recognition_gap_folder = "/gpfs01/bethge/home/jborowski/CHAM_recognition_gap/JOV_publication_git_bethgelab/recognition_gap/"

## Load libraries

In [ ]:

# custom imports
sys.path.insert(1, path_to_recognition_gap_folder)

## Data

Magnitude and standard deviation of recognition gap for **machine
algorithms** and **humans**. The values for the experiments with BagNets
correspond to the experiments with joint classes ("jointClasses") and
corner crops ("Ullman4"). The data for the experiments with machine
algorithms is taken from Ullman et al. (2016).

In [ ]:
# initialize dictionaries that will be plotted
recognitionGapsMachine = {}
recognitionGapsMachineStd = {}

In [ ]:
# get data from csv files for the experiments with BagNet
for exp_dir_condition, exp_dir_list in data_csv_utils.exp_dir_dict.items():
    if exp_dir_condition in ["Ullman_Ullman4_jointClasses", "IN_Ullman4"]:
        # get all the data in one dataframe
        all_data_df = data_csv_utils.get_df_from_exp_dir_list(exp_dir_list)
        # clean the data such that only data from images which yielded MIRCs is
        # contained
        all_data_df_real_MIRCs = data_csv_utils.get_df_with_data_from_real_MIRCs_only(
            all_data_df)

        # calculate the metrics that are displayed in the appendix:
        # A. mean and standard deviation of rec_gap
        recognitionGapsMachine[exp_dir_condition] = all_data_df_real_MIRCs.mean(
            axis=0).rec_gap
        recognitionGapsMachineStd[exp_dir_condition] = all_data_df_real_MIRCs.std(
            axis=0, ddof=0).rec_gap

In [ ]:
recognitionGapsMachine["machines_Ullman"] = 0.14
recognitionGapsMachineStd["machines_Ullman"] = 0.24

## Helper functions

In [ ]:
def plot_vertical_bar(this_key, x_position, this_label=""):
    """Plot the vertical bars corresponding to the machine data

    Args:
        this_key:   string to specify experiment
        x_position: position on x-axis
        this_label: optional label
    """
    plt.bar(
        x_position,
        recognitionGapsMachine[this_key],
        plot_utils.width,
        yerr=recognitionGapsMachineStd[this_key],
        color="white",
        edgecolor="k",
        label=this_label
    )

In [ ]:
def customize_axes(ax):
    """Adjust the axes.

    Args:
        ax: axes of plot
        """
    ax.set_ylabel("recognition gap")
    ax.yaxis.label.set_color("red")
    ax.set_xticks(list(range(len(recognitionGapsMachine))))
    ax.set_xticklabels(
        (
            "BagNet-33\non data from\nUllman et al.",
            "BagNet-33\non ImageNet\nsubset",
            "algorithms from\nUllman et al.\non their data"
        ),
    )
    ax.set_xlim(-0.5, len(recognitionGapsMachine) - 1 + 0.5)

    plot_utils.hide_right_and_top_spine(ax)

In [ ]:
def customize_legend():
    """Adjust the legend"""
    legend = plt.legend(bbox_to_anchor=(1.1, 1.05))
    # set the color of "recognition\ngap" in the legend red
    for i, text in enumerate(legend.get_texts()):
        if i == 1:
            text.set_color("red")

## Plot it!

In [ ]:
plt.figure(figsize=[5, 4.5])

# plot horizontal bar for recognition gaps of humans
plot_utils.plot_human_rec_gap_as_horizonal_bar(len(recognitionGapsMachine))

# plot vertical bars for recognition gaps of machines
plot_vertical_bar("Ullman_Ullman4_jointClasses", 0, "recognition\ngap")
plot_vertical_bar("IN_Ullman4", 1)
plot_vertical_bar("machines_Ullman", 2)

customize_axes(plt.axes())
customize_legend()

# save the figure and display it
plt.savefig("JOV_main_bar.svg", bbox_inches="tight")
plt.show()